**Data Analysis of Neighborhood in Toronto

**1. Part one 

First, import the necessary packages

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import geocoder
import folium
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

ModuleNotFoundError: No module named 'bs4'

Download the data and view it.

In [9]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
soup
# get tables
table = soup.find_all('table')[0] 
lst=table.find_all("td")
len(lst)
lst

NameError: name 'BeautifulSoup' is not defined

Create a new dataframe.

In [ ]:
# define the dataframe columns
column_names = ['Postcode','Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)
i=0

Loop through the <td> tags and get the text.

In [ ]:

while i <= 863:
    postcode=lst[i].get_text()
    borough=lst[i+1].get_text()
    neighborhood=lst[i+2].get_text().rstrip()
    df=df.append({'Postcode':postcode,'Borough':borough,'Neighborhood':neighborhood},ignore_index=True)
    i=i+3



Drop the columns with "Not assigned"

In [ ]:
df=df.drop(index=(df.loc[(df['Borough']=='Not assigned')].index))
df=df.groupby(['Postcode','Borough'],sort=False).agg(lambda x: ','.join(x))
print(df.head())

Count the number of rows

In [ ]:
df.shape

**2. Part two

Download the data of postcode and transfer it into a dataframe

In [ ]:
df_location=pd.read_csv("/Users/zhaowenchuan/Desktop/Geospatial_Coordinates.csv")

df_location.rename(index=str,columns={"Postal Code":"Postcode"},inplace=True)
df_location

Merge the two dataframes on Postcode

In [ ]:
location=pd.merge(df,df_location,on='Postcode')
location.head()

**3. Part three

First, we need to download the geograpical coordinate of Toronto

In [3]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))


NameError: name 'Nominatim' is not defined

Then we can map the cluster of neighborhood.

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=12)
# add markers to map
for lat, lng, borough, neighborhood in zip(location['Latitude'], location['Longitude'], location['Brought'], location['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto